In [15]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd


# Load Data

In [22]:
# Load the CSV file into a DataFrame
df_de = pd.read_csv('../Preprocessing/data_clean/de_clean_condensed.csv')

### Fix DE

In [17]:
#for DE datasate drop empty lines
df_de = df_de.dropna(subset=['data'])

# Sentiment analysis full text

In [23]:
def perform_sentiment_analysis(df):
    # Load the tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained("oliverguhr/german-sentiment-bert")
    model = AutoModelForSequenceClassification.from_pretrained("oliverguhr/german-sentiment-bert")

    # Create an empty list to store the sentiment scores
    sentiment_scores = []

    # Iterate over the 'data' column in the DataFrame
    for text in df['data']:
        # Tokenize the input text
        tokens = tokenizer.encode_plus(text, padding="max_length", truncation=True, max_length=128,
                                       return_tensors="pt")

        # Perform the sentiment analysis
        with torch.no_grad():
            logits = model(**tokens)[0]

        # Convert logits to predicted label (positive/negative)
        predicted_label = torch.argmax(logits, dim=1).item()
        sentiment = "positiv" if predicted_label == 1 else "negativ"

        # Append the sentiment score to the list
        sentiment_scores.append(sentiment)

    # Add the sentiment scores as a new column in the DataFrame
    df['sentiment'] = sentiment_scores

    return df


# Perform sentiment analysis on the DataFrame
df = perform_sentiment_analysis(df_de)

# Print the updated DataFrame
print(df)

                                                  data             player  \
0    trainer alonso mitgereisten fans monaco glucks...  exequiel palacios   
1    sechser droht viereinhalb monaten pause bitter...  exequiel palacios   
2    bayerleverkusen sechser erklart budapest spiel...  exequiel palacios   
3    bakker mittwoch abschluss training bayerleverk...  exequiel palacios   
4    allerdings andrich argentinischen weltmeister ...  exequiel palacios   
..                                                 ...                ...   
605  bayerleverkusen wehrt europaleague lange gut s...     piero hincapie   
606  alonso taktik duell alten lehrmeister mourinho...     piero hincapie   
607  trotz pleite halbfinal hinspiel rom gibt bayer...     piero hincapie   
608  hitzigen aufeinandertreffen stadio olimpico na...     piero hincapie   
609  hitzigen aufeinandertreffen stadio olimpico na...     piero hincapie   

    language           publishedAt sentiment  
0         de  2023-02-24T09:

In [24]:
df[df['sentiment'] == 'negativ'].shape[0]/df.shape[0]

0.8901639344262295

In [19]:
# Drop rows where 'Label' is NaN or empty
df_de.dropna(subset=['Label'], inplace=True)

# Print the updated DataFrame
data = df_de[df_de['Label'] != 'neutral']

df_de

,data,player,language,publishedAt,Label,sentiment
14,"zweimal verwandelte palacios, es wurde ein ube...",palacios,de,2023-03-19T20:01:45Z,positiv,negativ
23,schalke: reis nimmt den spieler ausdrucklich i...,palacios,de,2023-04-03T08:12:20Z,neutral,negativ
31,palacios (r.) verletzte sich leicht im hinspie...,palacios,de,2023-04-17T06:50:19Z,neutral,negativ
41,"""auf jeden fall"" sei das ein ganz grosser tag,...",palacios,de,2023-04-20T20:54:03Z,positiv,negativ
58,beim abschlusstraining trug letzterer zumindes...,palacios,de,2023-05-17T11:20:50Z,neutral,negativ
63,bayerleverkusen argentinischer weltmeister pal...,palacios,de,2023-05-14T18:07:21Z,negativ,negativ
83,"""es gibt im moment nicht viel besseres in der ...",frimpong,de,2023-03-31T07:59:57Z,positiv,negativ
93,der fc bayern munchen interessiert sich nach i...,frimpong,de,2023-04-06T16:49:00Z,neutral,negativ
95,frimpong hat noch bis vertrag in bayerleverkus...,frimpong,de,2023-04-07T07:42:27Z,neutral,negativ
102,hincapie (l.) und tah bedanken sich fur die fa...,frimpong,de,2023-04-14T08:16:22Z,positiv,negativ


In [20]:
data

,data,player,language,publishedAt,Label,sentiment
14,"zweimal verwandelte palacios, es wurde ein ube...",palacios,de,2023-03-19T20:01:45Z,positiv,negativ
41,"""auf jeden fall"" sei das ein ganz grosser tag,...",palacios,de,2023-04-20T20:54:03Z,positiv,negativ
63,bayerleverkusen argentinischer weltmeister pal...,palacios,de,2023-05-14T18:07:21Z,negativ,negativ
83,"""es gibt im moment nicht viel besseres in der ...",frimpong,de,2023-03-31T07:59:57Z,positiv,negativ
102,hincapie (l.) und tah bedanken sich fur die fa...,frimpong,de,2023-04-14T08:16:22Z,positiv,negativ
123,bayerleverkusen torschutze amiri (rechts) u...,frimpong,de,2023-04-23T17:33:57Z,positiv,negativ
124,frimpong bekommt gluckwunsche seines trainers ...,frimpong,de,2023-04-23T18:00:31Z,positiv,negativ
130,selbst als stammkraft frimpong in den vergange...,frimpong,de,2023-05-23T10:24:42Z,negativ,negativ
173,schalke wiederum hatte durch michael frey ein ...,tah,de,2023-04-01T15:44:09Z,positiv,negativ
199,"alonso ""stolz"": werkself seit spielen ungeschl...",tah,de,2023-04-23T20:48:00Z,positiv,negativ


In [21]:
# Calculate the accuracy
accuracy = (data['sentiment'] == data['Label']).mean() * 100

# Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy))

Accuracy: 50.00%
